# 10 - Extracción de características y similitud de oraciones (Embeddings)

<br>
<br>

<img src="https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/embeddings.webp" style="width:400px;"/>

<h1>Tabla de Contenidos<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1---Modelos-de-extracción-de-características-(Embeddings)" data-toc-modified-id="1---Modelos-de-extracción-de-características-(Embeddings)-1">1 - Modelos de extracción de características (Embeddings)</a></span></li><li><span><a href="#2---Pipeline-de-Transformers-para-Embeddings" data-toc-modified-id="2---Pipeline-de-Transformers-para-Embeddings-2">2 - Pipeline de Transformers para Embeddings</a></span></li><li><span><a href="#3---Tokenizador" data-toc-modified-id="3---Tokenizador-3">3 - Tokenizador</a></span></li><li><span><a href="#4---Transformadores-de-frases" data-toc-modified-id="4---Transformadores-de-frases-4">4 - Transformadores de frases</a></span></li><li><span><a href="#5---Similitud-de-oraciones" data-toc-modified-id="5---Similitud-de-oraciones-5">5 - Similitud de oraciones</a></span></li></ul></div>

## 1 - Modelos de extracción de características (Embeddings)

Los modelos de extracción de características, modelos de embedding o modelos de incrustación son técnicas fundamentales en el campo del NLP, que permiten representar datos como vectores en un espacio de características continuo. Estos modelos capturan la semántica de los datos y facilitan el manejo de relaciones complejas dentro de grandes conjuntos de datos. Algunos de los modelos de embedding más conocidos son:


1. **Word2Vec**: Desarrollado por Google, es uno de los modelos de embedding de palabras más populares. Utiliza redes neuronales para aprender representaciones vectoriales de palabras a partir de grandes corpora de texto. Tiene dos arquitecturas principales: CBOW (Continuous Bag of Words) y Skip-gram.

2. **GloVe** (Global Vectors for Word Representation): Propuesto por investigadores de Stanford, GloVe es un método que genera embeddings de palabras al factorizar matrices que describen las co-ocurrencias estadísticas de palabras en un corpus. Se diferencia de Word2Vec en que no solo se basa en contextos locales de palabras sino en estadísticas globales del corpus.

3. **FastText**: Desarrollado por Facebook Research, este modelo extiende Word2Vec para considerar no solo palabras completas sino también sub-palabras o n-gramas de caracteres. Esto permite que el modelo maneje mejor palabras desconocidas o raras.

4. **BERT** (Bidirectional Encoder Representations from Transformers): Introducido por Google AI, BERT usa la arquitectura de Transformer para pre-entrenar embeddings de palabras en contextos bidireccionales. Esto permite capturar un contexto más rico y mejorar el rendimiento en tareas de NLP downstream.

5. **ELMo** (Embeddings from Language Models): ELMo utiliza modelos de lenguaje basados en redes neuronales LSTM para aprender embeddings de palabras basados en el contexto de uso. Los embeddings resultantes son dinámicos, ajustándose según el contexto en el que aparecen las palabras.



Estos modelos de embedding son la base de los modelos que hemos venido utilizando anteriormente. Los tokenizadores son fundamentalmente extractores de características, pues convierten los textos en vectores, que posteriormente son introducidos en los modelos de transformers. Recordemos que el tokenizador primero convierte el texto en tokens, las piezas mínimas de texto, y más tarde convierte esos tokens a vectores. La dimensión de esos vectores viene determinado por el propio modelo de embedding. 

Existen modelos que devuelven vectores de 384, 768, 1024, 1536 o incluso 4096 elementos. Generalmente, un embedding de mayor dimensión puede capturar más información y matices sobre los datos. Esto puede resultar en una mejor capacidad para distinguir entre conceptos más sutiles o complejos. Sin embargo, usar embeddings de mayor dimensión incrementa la carga computacional. Esto significa más uso de memoria, tiempos de entrenamiento más largos, y puede requerir hardware más potente. Además, los embeddings de mayor dimensión pueden ser más propensos al overfitting, especialmente si el tamaño del dataset de entrenamiento es pequeño.

## 2 - Pipeline de Transformers para Embeddings

Veamos algunos de los modelos de embeddings que proporciona Hugging Face. En primer lugar, [gte-small](https://huggingface.co/Supabase/gte-small), un modelo que nos devuelve un vector de 384 elementos para cada token y que tiene un peso de 70Mb. GTE significa General Text Embeddings, y son modelos entrenados por la [Academia DAMO de Alibaba](https://www.alibabagroup.com/en-US/about-alibaba-businesses-1496657217451982848). Se basan principalmente en el marco de trabajo de BERT y actualmente ofrecen tres tamaños diferentes de modelos, incluyendo GTE-large (1024), GTE-base (768) y GTE-small (384). Los modelos GTE están entrenados en un corpus de gran escala de pares de texto relevantes, cubriendo una amplia gama de dominios y escenarios. Esto permite que los modelos GTE se apliquen a diversas tareas de procesamiento de texto, incluyendo la recuperación de información, la similitud textual semántica y la reordenación de textos. Vamos a cargarlo con el pipeline y ver su respuesta:

In [1]:
from transformers import pipeline

In [2]:
tarea = 'feature-extraction'

modelo = 'Supabase/gte-small'

In [3]:
gte_pipe = pipeline(task=tarea, model=modelo)

/opt/anaconda3/envs/ia/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [4]:
frase = 'hola, buenos dias'

In [6]:
gte_pipe(frase, return_tensors=True).shape

torch.Size([1, 8, 384])

In [7]:
frase = 'hola, buenos dias, ¿como estas?. Yo bien ¿y tu?. Aqui estamos'

gte_pipe(frase, return_tensors=True).shape

torch.Size([1, 27, 384])

Como vemos, las dimensiones del tensor han cambiado. El `1` se refiere a la secuencia, el `384` es la dimensión del vector por token. El `8` o el `27` se refiere al número de tokens. Este modelo tiene un máximo de 512 tokens, por lo que la dimensión máxima del tensor es `[1, 512, 384]`.

Como dijimos, existen modelos de embedding que nos devuelven vectores con más dimensiones. Facebook tiene un modelo basado en BART, llamado [bart-base](https://huggingface.co/facebook/bart-base). Este modelo devuelve un vector de 768 elementos y pesa aproximadamente 560Mb.

In [8]:
modelo = 'facebook/bart-base'

In [9]:
bart_pipe = pipeline(task=tarea, model=modelo)

/opt/anaconda3/envs/ia/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [10]:
frase = 'hola, buenos dias'

bart_pipe(frase, return_tensors=True).shape

torch.Size([1, 9, 768])

In [11]:
frase = 'hola, buenos dias, ¿como estas?. Yo bien ¿y tu?. Aqui estamos'

bart_pipe(frase, return_tensors=True).shape

torch.Size([1, 32, 768])

Vemos que el número de tokens de este modelo también es distinta, de 8 se pasó a 9 y de 27 se pasó a 34, además de tener los 768 elementos por token frente a los 384 del modelo anterior. 

## 3 - Tokenizador

Hemos visto en los capítulos anteriores, que este proceso de embedding es el que realizan los tokenizadores. Dado un texto, el tokenizador extrae los tokens y luego vectoriza. Veamos este último modelo, bart-base, usado a través del AutoTokenizer de transformers.

In [12]:
from transformers import AutoTokenizer

In [13]:
tokenizador = AutoTokenizer.from_pretrained(modelo)

In [14]:
tokenizador

BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}

Ya hemos visto este tokenizador, BartTokenizerFast. Es una implementación específica de un tokenizador para el modelo BART, optimizada que aprovecha la biblioteca Rust tokenizers para ofrecer un rendimiento más rápido en comparación con su versión en Python. Ofrece un mejor rendimiento sin sacrificar la calidad o la precisión de la tokenización, lo cual es crucial para la eficacia de los modelos de lenguaje en tareas de NLP.


In [15]:
frase = 'hola, buenos dias'

In [16]:
respuesta = tokenizador(frase, return_tensors='pt')

respuesta

{'input_ids': tensor([[    0,  9649,   102,     6, 10306, 36048,  2269,   281,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [18]:
respuesta['input_ids'].shape

torch.Size([1, 9])

In [19]:
frase = 'hola, buenos dias, ¿como estas?. Yo bien ¿y tu?. Aqui estamos'

respuesta = tokenizador(frase, return_tensors='pt')

respuesta['input_ids'].shape

torch.Size([1, 32])

Las salidas del tokenizador tiene dimensiones distintas de las que devuelve el pipeline, solo el 1 de la secuencia y el número de tokens. Así pues el tokenizador saca un vector distinto, pero que básicamente significa los mismo. Veamos cuales son los tokens:

In [20]:
tokenizador.batch_decode(respuesta['input_ids'])

['<s>hola, buenos dias, ¿como estas?. Yo bien ¿y tu?. Aqui estamos</s>']

In [21]:
print(respuesta.tokens())

['<s>', 'hol', 'a', ',', 'Ġbu', 'enos', 'Ġdi', 'as', ',', 'ĠÂ', '¿', 'com', 'o', 'Ġest', 'as', '?', '.', 'ĠYo', 'Ġb', 'ien', 'ĠÂ', '¿', 'y', 'Ġtu', '?', '.', 'ĠAqu', 'i', 'Ġest', 'am', 'os', '</s>']


In [22]:
len(respuesta.tokens())

32

La dimensión del vector que sale del tokenizador coincide con el número de tokens que genera, es decir, tenemos un número para cada token generado.

## 4 - Transformadores de frases

Ahora vamos a transformar frases completas a un vector. Usaremos el modelo [all-roberta-large-v1](https://huggingface.co/sentence-transformers/all-roberta-large-v1) un modelo de sentence-transformers que mapea oraciones y párrafos a un espacio vectorial de 1024 dimensiones y puede ser utilizado para tareas como el agrupamiento o la búsqueda semántica. Tiene un peso aproximado de 1.5Gb.

Usamos el objeto `SentenceTransformer`, que se utiliza para convertir oraciones y párrafos en vectores densos de alta dimensión, lo que permite comparar y medir la similitud semántica entre ellos. Vector denso se refiere a que el vector de salida no tiene ningún elemento 0. 

In [27]:
modelo = 'sentence-transformers/all-roberta-large-v1'

In [28]:
from sentence_transformers import SentenceTransformer

In [29]:
transformador = SentenceTransformer(modelo)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
transformador

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

**Componentes del SentenceTransformer**

1. **Transformer**: Este es el primer módulo en la cadena del SentenceTransformer y utiliza un modelo preentrenado de Transformer, específicamente RobertaModel. Aquí es donde se realiza la primera etapa de procesamiento del texto.

    + max_seq_length: Define la longitud máxima de la secuencia de entrada que el modelo puede manejar. En este caso, está configurado para 256 tokens.
    + do_lower_case: Indica si el texto de entrada debe convertirse a minúsculas antes de procesarlo. Aquí está configurado como False, lo que significa que el texto se procesará tal como se recibe.


2. **Pooling**: Este módulo maneja la forma en que se combinan o resumen las representaciones de tokens individuales obtenidas del modelo RobertaModel para formar un único vector de embedding para toda la oración.

    + word_embedding_dimension: La dimensión de los embeddings de cada palabra, en este caso, 1024.
    + pooling_mode_cls_token: Si se usa, toma el token CLS, usado a menudo como un resumen de toda la entrada en modelos como BERT y RoBERTa, como el embedding de la oración. Aquí está desactivado.
    + pooling_mode_mean_tokens: Calcula la media de todos los embeddings de tokens para obtener el embedding de la oración. Está activado, lo que significa que esta es la técnica de pooling utilizada.
    + pooling_mode_max_tokens, pooling_mode_mean_sqrt_len_tokens, pooling_mode_weightedmean_tokens, pooling_mode_lasttoken: Otras técnicas de pooling que están disponibles pero no se utilizan en esta configuración.
    + include_prompt: Específico para si se incluye o no algún prompt o guía en el cálculo del embedding.


3. **Normalize**: Este último paso en la cadena del SentenceTransformer normaliza el vector de embedding resultante. La normalización suele implicar escalar el vector de modo que su norma sea 1. Esto es útil para muchas tareas de comparación de vectores, como la similitud coseno, ya que simplifica los cálculos y a menudo mejora el rendimiento del modelo en tareas de similitud o clasificación.


In [31]:
frase = 'hola, buenos dias'

In [33]:
transformador.encode(frase).shape

(1024,)

In [34]:
frase = 'hola, buenos dias, ¿como estas?. Yo bien ¿y tu?. Aqui estamos'

transformador.encode(frase).shape

(1024,)

Se puede observar que todas las frases, sea cual sea su número de tokens, el modelo lo convierte a un vector de 1024 elementos. En realidad realiza la media por token, según su configuración por defecto. Veamos que ocurre cuando le damos una lista de frases:

In [35]:
frases = ['hola alegre', 'estamos en clase de IA', 'estamos aprendiendo embeddings']

In [36]:
transformador.encode(frases).shape

(3, 1024)

El transformador nos devuelve un vector para cada frase con 1024 elementos. Veamos algunos casos de uso, fuera de los modelos que ya hemos visto, de esta transformación del texto.

## 5 - Similitud de oraciones

Uno de los usos habituales de los embeddings es comparar dos frases, cómo de parecidas son. La métrica que se suele usar para ello es la similitud del coseno. La similitud del coseno es una medida que se utiliza para calcular el grado de similitud entre dos vectores en un espacio multidimensional, ignorando su magnitud. Se calcula de la siguiente manera:

$$coseno (\vec{a}, \vec{b}) = \frac{\vec{a} · \vec{b}}{|\vec{a}| · |\vec{b}|}$$


siendo:

+ $\vec{a}, \vec{b}$: vectores 
+ $\vec{a} · \vec{b}$: producto escalar de $\vec{a}$ y $\vec{b}$
+ $|\vec{a}|$,  $|\vec{b}|$: normas de los vectores (magnitudes)


<br>

**Características de la similitud del coseno**:

1. Rango de Valores: La similitud del coseno puede tomar valores entre -1 y 1.

    + 1 indica que los dos vectores son idénticos en orientación.
    + 0 indica que los vectores son ortogonales (independientes).
    + -1 indica que los vectores son diametralmente opuestos.


2. Insensible a la Magnitud: Solo importa el ángulo entre los vectores, no su longitud. Esto es útil cuando las magnitudes no llevan información relevante o cuando solo la dirección de los vectores es importante.



Por supuesto, existen otra maneras de calcular la similitud entre dos vectores, usando otras [métricas de distancias](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html), aunque si no existe un criterio específico, siempre se usa la similitud del coseno.

Calcularemos la similitud del coseno con scikit-learn. Es posible que necesitemos ejecutar el siguiente comando por terminal para instalar la librería:

```bash
pip install scikit-learn
```

In [38]:
from sklearn.metrics.pairwise import cosine_similarity

In [39]:
help(cosine_similarity)

Help on function cosine_similarity in module sklearn.metrics.pairwise:

cosine_similarity(X, Y=None, dense_output=True)
    Compute cosine similarity between samples in X and Y.
    
    Cosine similarity, or the cosine kernel, computes similarity as the
    normalized dot product of X and Y:
    
        K(X, Y) = <X, Y> / (||X||*||Y||)
    
    On L2-normalized data, this function is equivalent to linear_kernel.
    
    Read more in the :ref:`User Guide <cosine_similarity>`.
    
    Parameters
    ----------
    X : {array-like, sparse matrix} of shape (n_samples_X, n_features)
        Input data.
    
    Y : {array-like, sparse matrix} of shape (n_samples_Y, n_features),             default=None
        Input data. If ``None``, the output will be the pairwise
        similarities between all samples in ``X``.
    
    dense_output : bool, default=True
        Whether to return dense output even when the input is sparse. If
        ``False``, the output is sparse if both input arr

In [40]:
frases = ['hola alegre', 'hola galan']

In [41]:
vectores = transformador.encode(frases)

In [43]:
vectores.shape

(2, 1024)

In [45]:
cosine_similarity([vectores[0]], [vectores[1]])

array([[0.5365212]], dtype=float32)

In [46]:
cosine_similarity([vectores[1]], [vectores[0]])

array([[0.5365212]], dtype=float32)

In [47]:
cosine_similarity([vectores[0]], [vectores[0]])

array([[1.]], dtype=float32)

Desde aquí, ya tenemos un métrica de como de parecidas son dos frases, tanto más parecidas cuanto más cercano a 1 sea la similitud del coseno, y tanto más discordantes cuanto más cercano a -1 sea dicha similitud. Si la similitud está en torno a 0, esas frases no se parecen en nada.